<a href="https://colab.research.google.com/github/yonabell/Report/blob/main/Reporting_pythond_code_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
import pandas as pd
import os

In [54]:
from google.colab import files
uploaded = files.upload()

Saving 2024_sample_data.xlsx to 2024_sample_data (3).xlsx


In [55]:
file_path = '2024_sample_data.xlsx'

In [56]:
def clean_data(file_path):
    # Load the data with no predefined header
    cpt_df = pd.read_excel(file_path, engine='openpyxl', header=4)

    # Filtering rows and extracting only 'A' followed by nine digits
    filtered_cpt_df = cpt_df[cpt_df['Unnamed: 0'].str.extract(r'(A\d{9})', expand=False).notna()].copy()

    # Extracting the ID
    filtered_cpt_df['ID'] = filtered_cpt_df['Unnamed: 0'].str.extract(r'(A\d{9})', expand=False)

    # Reorder columns to place 'ID' next to 'Unnamed: 0'
    cols = filtered_cpt_df.columns.tolist()
    id_index = cols.index('ID')
    cols = cols[:1] + [cols[id_index]] + cols[1:id_index] + cols[id_index + 1:]
    filtered_cpt_df = filtered_cpt_df[cols]

    # Drop 'Unnamed: 0' column
    filtered_cpt_df = filtered_cpt_df.drop('Unnamed: 0', axis=1)

    # Drop specific columns (1 to 7)
    filtered_cpt_df = filtered_cpt_df.drop(filtered_cpt_df.columns[1:8], axis=1)

    # Select every 7th column starting from the 8th column (index 7)
    column_names = filtered_cpt_df.columns.tolist()
    columns_to_drop = column_names[7::7]

    # Drop the selected columns from the DataFrame
    filtered_cpt_df.drop(columns=columns_to_drop, inplace=True)

    return filtered_cpt_df

In [57]:
# Call the clean_data function
filtered_cpt_df1 = clean_data(file_path)

In [58]:
# Display the filtered_cpt_df
filtered_cpt_df1

,ID,Purch. Volume.1,S-Revenues.1,Profit Margin I.1,PM I % of Purchased Volume.1,Profit Margin I*.1,PM I* % of Purchased Volume.1,Purch. Volume.2,S-Revenues.2,Profit Margin I.2,PM I % of Purchased Volume.2,Profit Margin I*.2,PM I* % of Purchased Volume.2,Purch. Volume.3,S-Revenues.3,Profit Margin I.3,PM I % of Purchased Volume.3,Profit Margin I*.3,PM I* % of Purchased Volume.3
5,A987654321,678.0,87464.0,7656.0,44.68,454.0,0.09,4355.0,43.0,2.0,0.14,546.0,3.785323,65.0,34.0,34.0,9.8384,45.0,2.033
8,A123456789,8876.0,3566.0,765.0,71.44,4326.0,0.12,1345.0,33.0,32.0,0.20,65.0,5.724400,543.0,725.0,643.0,15.7322,542.0,2.519


In [59]:
def aggregate_data(filtered_df):
    # Columns to be aggregated (excluding 'ID')
    agg_cols = ['Purch. Volume', 'S-Revenues', 'Profit Margin I',
                'PM I % of Purchased Volume', 'Profit Margin I*',
                'PM I* % of Purchased Volume']

    # Get all columns that start with the names in agg_cols
    cols_to_convert = [col for col in filtered_df.columns if any(col.startswith(agg_col) for agg_col in agg_cols)]

    # Convert the selected columns to numeric
    filtered_df[cols_to_convert] = filtered_df[cols_to_convert].apply(pd.to_numeric, errors='coerce')

    # Create a new DataFrame to store the aggregated results
    aggregated_df = pd.DataFrame(columns=['ID'] + agg_cols)

    # Group the DataFrame by 'ID' and aggregate
    for id_val, group in filtered_df.groupby('ID'):
        new_row = {'ID': id_val}
        for agg_col in agg_cols:
            new_row[agg_col] = group[[col for col in group.columns if col.startswith(agg_col)]].sum(axis=1).sum()  # Sum across rows and then sum the results
        aggregated_df = pd.concat([aggregated_df, pd.DataFrame([new_row])], ignore_index=True)

    return aggregated_df


In [60]:
# Call the clean_data function
aggregated_df1 = aggregate_data(filtered_cpt_df1)

<ipython-input-59-240140c92c73>:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  aggregated_df = pd.concat([aggregated_df, pd.DataFrame([new_row])], ignore_index=True)


In [61]:
# Display the aggregated_df
aggregated_df1

,ID,Purch. Volume,S-Revenues,Profit Margin I,PM I % of Purchased Volume,Profit Margin I*,PM I* % of Purchased Volume
0,A123456789,10764.0,4324.0,6373.0,87.3722,4933.0,8.363400
1,A987654321,5098.0,87541.0,8737.0,54.6584,1045.0,5.908323


In [62]:
def rename_columns(aggregated_df):
    # Define the new column names
    new_column_names = {
        'ID': 'BPID',
        'Purch. Volume': 'PurchVolume',
        'S-Revenues': 'Revenue',
        'Profit Margin I': 'PMI',
        'PM I % of Purchased Volume': 'PMIRate',
        'Profit Margin I*': 'PMIStar',
        'PM I* % of Purchased Volume': 'PMIStarRate'
    }

    # Rename the columns
    aggregated_df = aggregated_df.rename(columns=new_column_names)

    # Return the modified DataFrame
    return aggregated_df

In [63]:
# call the rename function
aggregated_df1 = rename_columns(aggregated_df1)

In [64]:
# Display the aggregated_df
aggregated_df1

,BPID,PurchVolume,Revenue,PMI,PMIRate,PMIStar,PMIStarRate
0,A123456789,10764.0,4324.0,6373.0,87.3722,4933.0,8.363400
1,A987654321,5098.0,87541.0,8737.0,54.6584,1045.0,5.908323


In [65]:
from google.colab import files
uploaded = files.upload()

Saving 2023_sample_data.xlsx to 2023_sample_data (2).xlsx


In [66]:
file_path = '2023_sample_data.xlsx'

In [67]:
# Call the clean_data function
filtered_cpt_df2 = clean_data(file_path)

In [68]:
# Create a dictionary to store the month-wise column mappings
month_columns = {}
months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

for i, month in enumerate(months):
  start_col_index = 1 + (i * 6)  # Starting column index for the month
  end_col_index = start_col_index + 6
  month_columns[month] = filtered_cpt_df.columns[start_col_index:end_col_index].tolist()

In [69]:
# Define the months you want to filter for (dynamically)
filtered_months = ['February', 'March']  # Example: Change this list as needed

# Create a list to store the selected columns
selected_columns = ['ID']

# Loop through the filtered months and add corresponding columns to the list
for month in filtered_months:
  if month in month_columns:
    selected_columns.extend(month_columns[month])

# Filter the DataFrame with selected columns
filtered_df2 = filtered_cpt_df2[selected_columns]

In [70]:
# Display the filtered DataFrame
filtered_df2

,ID,Purch. Volume.2,S-Revenues.2,Profit Margin I.2,PM I % of Purchased Volume.2,Profit Margin I*.2,PM I* % of Purchased Volume.2,Purch. Volume.3,S-Revenues.3,Profit Margin I.3,PM I % of Purchased Volume.3,Profit Margin I*.3,PM I* % of Purchased Volume.3
5,A987654321,4355.0,43.0,2.0,0.14,546.0,3.785323,65.0,34.0,34.0,9.8384,45.0,2.033
8,A123456789,1345.0,33.0,32.0,0.20,65.0,5.724400,543.0,725.0,643.0,15.7322,542.0,2.519


In [71]:
# Call the clean_data function
aggregated_df2 = aggregate_data(filtered_df2)

<ipython-input-59-240140c92c73>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df[cols_to_convert] = filtered_df[cols_to_convert].apply(pd.to_numeric, errors='coerce')
<ipython-input-59-240140c92c73>:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  aggregated_df = pd.concat([aggregated_df, pd.DataFrame([new_row])], ignore_index=True)


In [72]:
aggregated_df2

,ID,Purch. Volume,S-Revenues,Profit Margin I,PM I % of Purchased Volume,Profit Margin I*,PM I* % of Purchased Volume
0,A123456789,1888.0,758.0,1282.0,15.9322,607.0,8.243400
1,A987654321,4420.0,77.0,627.0,9.9784,591.0,5.818323


In [73]:
# call the rename function
aggregated_df2 = rename_columns(aggregated_df2)

In [79]:
aggregated_df1

,BPID,PurchVolume,Revenue,PMI,PMIRate,PMIStar,PMIStarRate
0,A123456789,10764.0,4324.0,6373.0,87.3722,4933.0,8.363400
1,A987654321,5098.0,87541.0,8737.0,54.6584,1045.0,5.908323


In [76]:
aggregated_df2

,BPID,PurchVolume,Revenue,PMI,PMIRate,PMIStar,PMIStarRate
0,A123456789,1888.0,758.0,1282.0,15.9322,607.0,8.243400
1,A987654321,4420.0,77.0,627.0,9.9784,591.0,5.818323


In [77]:
# prompt: Now I want to aggregate(sum) aggregated_df1 and aggregated_df2

aggregated_df_combined = pd.concat([aggregated_df1, aggregated_df2], ignore_index=True)
aggregated_df_sum = aggregated_df_combined.groupby('BPID').sum().reset_index()

In [78]:
aggregated_df_sum

,BPID,PurchVolume,Revenue,PMI,PMIRate,PMIStar,PMIStarRate
0,A123456789,12652.0,5082.0,7655.0,103.3044,5540.0,16.606799
1,A987654321,9518.0,87618.0,9364.0,64.6368,1636.0,11.726646
